In [1]:
from langchain_chroma import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
import os
import requests
import json
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

### Load contents from Notion

In [2]:
# Notion 인증 정보
NOTION_TOKEN = "ntn_S49134845636QN7OizYlyythCTORUXOCvYcp2U19S0P6dy"
NOTION_VERSION = "2022-06-28"

headers = {
    "Authorization": f"Bearer {NOTION_TOKEN}",
    "Notion-Version": NOTION_VERSION,
    "Content-Type": "application/json"
}

def get_page_content(page_id):
    """
    페이지 ID를 기반으로 페이지 내용을 가져옵니다.

    Args:
        page_id (str): Notion 페이지 ID (대시 제외)

    Returns:
        dict: 페이지 속성 및 내용
    """
    # 페이지 속성 가져오기
    page_url = f"https://api.notion.com/v1/pages/{page_id}"
    response = requests.get(page_url, headers=headers)

    if response.status_code != 200:
        print(f"오류 발생: {response.status_code}")
        print(response.text)
        return None

    page_data = response.json()

    # 페이지 내용(블록) 가져오기
    blocks_url = f"https://api.notion.com/v1/blocks/{page_id}/children"
    response = requests.get(blocks_url, headers=headers)

    if response.status_code != 200:
        print(f"블록 가져오기 오류: {response.status_code}")
        print(response.text)
        return page_data

    blocks_data = response.json()

    # 페이지 데이터에 블록 내용 추가
    page_data["content"] = blocks_data["results"]

    return page_data

def fetch_all_blocks(block_id):
    """
    블록의 모든 하위 블록을 재귀적으로 가져옵니다.

    Args:
        block_id (str): 블록 ID

    Returns:
        list: 모든 하위 블록 목록
    """
    blocks_url = f"https://api.notion.com/v1/blocks/{block_id}/children"
    response = requests.get(blocks_url, headers=headers)

    if response.status_code != 200:
        print(f"블록 가져오기 오류: {response.status_code}")
        print(response.text)
        return []

    blocks_data = response.json()["results"]

    # 하위 블록이 있는 블록 타입들
    has_children_types = [
        "paragraph", "bulleted_list_item", "numbered_list_item",
        "toggle", "child_page", "child_database", "column_list",
        "column", "table", "synced_block"
    ]

    all_blocks = []

    for block in blocks_data:
        all_blocks.append(block)

        # 하위 블록이 있는 경우 재귀적으로 가져오기
        if block.get("has_children") and block.get("type") in has_children_types:
            children = fetch_all_blocks(block["id"])
            # 하위 블록이 있는 경우에만 children 키 추가
            if children:
                block["children"] = children

    return all_blocks

def extract_text_content(blocks):
    """
    블록에서 텍스트 내용만 추출합니다. 이때, child_page의 제목을 구분자로 먼저 추가합니다.

    Args:
        blocks (list): 블록 목록

    Returns:
        str: 추출된 텍스트 내용
    """
    text_content = ""

    for block in blocks:
        block_type = block.get("type")

        # child_page의 경우, 제목 먼저 삽입
        if block_type == "child_page":
            child_title = block.get("child_page", {}).get("title", "")
            if child_title:
                text_content += f"\n\n### {child_title} ###\n\n"

        if block_type == "paragraph":
            rich_text = block.get("paragraph", {}).get("rich_text", [])
            for text in rich_text:
                text_content += text.get("plain_text", "")
            text_content += "\n\n"

        elif block_type == "heading_1":
            rich_text = block.get("heading_1", {}).get("rich_text", [])
            for text in rich_text:
                text_content += "# " + text.get("plain_text", "")
            text_content += "\n\n"

        elif block_type == "heading_2":
            rich_text = block.get("heading_2", {}).get("rich_text", [])
            for text in rich_text:
                text_content += "## " + text.get("plain_text", "")
            text_content += "\n\n"

        elif block_type == "heading_3":
            rich_text = block.get("heading_3", {}).get("rich_text", [])
            for text in rich_text:
                text_content += "### " + text.get("plain_text", "")
            text_content += "\n\n"

        elif block_type == "bulleted_list_item":
            rich_text = block.get("bulleted_list_item", {}).get("rich_text", [])
            text_content += "• "
            for text in rich_text:
                text_content += text.get("plain_text", "")
            text_content += "\n"

        elif block_type == "numbered_list_item":
            rich_text = block.get("numbered_list_item", {}).get("rich_text", [])
            text_content += "1. "
            for text in rich_text:
                text_content += text.get("plain_text", "")
            text_content += "\n"

        elif block_type == "to_do":
            rich_text = block.get("to_do", {}).get("rich_text", [])
            checked = block.get("to_do", {}).get("checked", False)
            text_content += "[{}] ".format("x" if checked else " ")
            for text in rich_text:
                text_content += text.get("plain_text", "")
            text_content += "\n"

        elif block_type == "code":
            rich_text = block.get("code", {}).get("rich_text", [])
            language = block.get("code", {}).get("language", "")
            text_content += f"```{language}\n"
            for text in rich_text:
                text_content += text.get("plain_text", "")
            text_content += "\n```\n\n"

        # 자식 블록이 있으면 재귀적으로 처리
        if "children" in block:
            text_content += extract_text_content(block["children"])

    return text_content



# 메인 실행 코드
def main():
    # 사용자로부터 페이지 ID 입력받기
    page_id = input("Notion 페이지 ID를 입력하세요: ")
    page_id = page_id.replace("-", "")  # 대시가 있으면 제거

    print("페이지 내용을 가져오는 중...")

    # 페이지 내용 가져오기
    page_data = get_page_content(page_id)

    if not page_data:
        print("페이지를 가져올 수 없습니다.")
        return

    # 모든 블록 가져오기 (재귀적으로)
    print("모든 블록을 재귀적으로 가져오는 중...")
    all_blocks = fetch_all_blocks(page_id)

    # 텍스트 내용 추출
    text_content = extract_text_content(all_blocks)

    # 결과 저장
    print("\n=== 페이지 기본 정보 ===")
    page_title = page_data.get("properties", {}).get("title", {}).get("title", [])
    if page_title:
        title_text = " ".join([text.get("plain_text", "") for text in page_title])
        print(f"제목: {title_text}")

    # JSON 형식으로 저장
    with open("documents/notion_page_content.json", "w", encoding="utf-8") as f:
        json.dump(page_data, f, ensure_ascii=False, indent=2)

    # 텍스트 형식으로 저장
    with open("documents/notion_page_content.txt", "w", encoding="utf-8") as f:
        f.write(text_content)

    print("\n페이지 내용이 'notion_page_content.json'과 'notion_page_content.txt'로 저장되었습니다.")

In [3]:
if __name__ == "__main__":
    main()

페이지 내용을 가져오는 중...
모든 블록을 재귀적으로 가져오는 중...

=== 페이지 기본 정보 ===
제목: 책 리뷰

페이지 내용이 'notion_page_content.json'과 'notion_page_content.txt'로 저장되었습니다.


### Split Text

In [4]:
import re

# 텍스트 로드 함수
def load_text_from_file(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        return file.read()

# 상위 구분자 ("### 제목 ###") 기준으로 분할
def split_text_with_headers(text):
    # "제목"으로 나누되, re.split은 제목과 내용이 번갈아 나옴
    chunks = re.split(r"### (.*?) ###", text)
    result = []
    for i in range(1, len(chunks), 2):  # 제목은 홀수 인덱스
        title = chunks[i].strip()
        content = chunks[i + 1].strip() if i + 1 < len(chunks) else ""
        result.append((title, content))
    return result

# 상위 구분자 기준 → 하위에서 RecursiveCharacterTextSplitter 적용
def chunk_text_with_recursive_splitter(text, chunk_size=500, chunk_overlap=50):
    header_chunks = split_text_with_headers(text)
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    final_chunks = []

    for title, content in header_chunks:
        # 제목을 붙여서 RecursiveCharacterTextSplitter에 넘김
        temp_doc = Document(page_content=content)
        sub_chunks = text_splitter.split_documents([temp_doc])
        for sub_chunk in sub_chunks:
            final_chunks.append(Document(page_content=sub_chunk.page_content))
    
    return final_chunks

# .txt 파일 경로
txt_file_path = "documents/notion_page_content.txt"

# 텍스트 로드
text = load_text_from_file(txt_file_path)

# 청킹 실행
texts = chunk_text_with_recursive_splitter(text, chunk_size=800)

In [5]:
# 출력
for i, chunk in enumerate(texts):
    num_chars = len(chunk.page_content)  # ✅ 글자 수 세기
    print("-" * 100)
    print(f"[{i+1}번째 청크] (글자 수: {num_chars})")
    print(chunk.page_content)

----------------------------------------------------------------------------------------------------
[1번째 청크] (글자 수: 791)
1️⃣ 한줄 평

쿠팡이 잘 나가고, 티메프가 왜 망했는지 알만하다

♓ Inuit Points ★★★☆☆

전자상거래는 떠오르는 산업이었지만, 이젠 성숙산업이 되어 버렸지요. 하지만 그 물밑은 살벌한 전쟁터입니다. 책은 결국 대형 플랫폼 위주로 편제될 수 밖에 없는 커머스 산업의 구조를 살피고, 거기에 대응하는 흐름을 짚어봅니다. 이 시대 커머스 산업을 단번에 훑기 편합니다. 별 셋 주었습니다.

❤️  To whom it matters

• 유통, 제조, 자영업에 종사하시는 분
• 자꾸 테무 광고가 뜨시는 분
🎢 Stories Related

• 큰 차원의 틀은 있지만, 13인 공저라 주장이 정교하거나 설득력 넘치진 않습니다.
• 편집에 공을 들여 챕터간 중복이 거의 없고 각자 관점을 잘 녹여 낸 점은 좋습니다.
• 어쩔수 없이 저자 별 식견의 차이는 있어, 챕터간 품질 차이는 확연합니다.
부제: 국경 없는 크로스보더 커머스 시대의 경쟁과 생존

🗨️ 좀 더 자세한 이야기

어디서 물건 사시나요?

전 요즘엔 쿠팡을 주로 쓰고, 홈플러스 오프라인+온라인으로 거의 모든 쇼핑을 합니다. 롱테일 제품은 네이버 스토어에서 검색을 하고, 아주 가끔 알리에 가며 테무는 들어갈 때마다 질려서 바로 나오곤 합니다.

책 읽다가 잊었던 예전이 떠올랐습니다. G마켓, 인터파크를 비롯해 여러 온라인 쇼핑몰을 썼고, 오프라인도 더 다양하게 다녔습니다. 하지만 이젠 딱 몇군데죠. 작지만 미묘한 변화는 산업적 함의가 큽니다.

책은 네가지 덩어리로 되어 있습니다.

1. 크로스보더 글로벌 플랫폼

2. 기술의 영향
--------------------------------------------------------------------------------------

### Save as vector DB with ChromaDB

In [6]:
embedding_model = HuggingFaceEmbeddings(model_name="jhgan/ko-sbert-sts")

vector_db = Chroma.from_documents(
    documents=texts,
    embedding=embedding_model,
    persist_directory="./chroma_db"
)

/var/folders/qj/sfy11gcx28d6_5n6tf2m8wc00000gn/T/ipykernel_66554/231741228.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="jhgan/ko-sbert-sts")


In [7]:
# load from disk
loaded_vector_db = Chroma(persist_directory="./chroma_db", embedding_function=embedding_model)

In [8]:
query = "'구글 임원에서 실리콘벨리 알바생이 되었습니다' 책에서 특히 인상 깊었던 육체노동의 경험은 어떤 것이었고, 그것이 주는 의미는 무엇이었을까요?"
results = loaded_vector_db.similarity_search(query, k=2)

for result in results:
    print(result.page_content)

### 구글 임원에서 실리콘밸리 알바생이 되었습니다 ###

1️⃣ 한줄 평

Tao of retiring: 은퇴를 생각하는 사이드 허슬러를 위한 안내서

♓ Inuit Points ★★★☆☆

구글 본사의 임원급으로 있다가 구조조정으로 나온 후, '1만명 만나기 프로젝트'를 시작합니다. 트레이더 조  점원, 스타벅스 바리스타, 리프트 기사와 펫시터 등 시간제 일을 하며 느낀 점을 이야기합니다. 생활비보다는 생활의 기운을 위해 시작한 육체노동의 가치를 말합니다. 이 감상은, 은퇴의 프리뷰이기도 해서 흥미롭습니다. 별 셋 주었습니다.

❤️  To whom it matters

• 평생 사무직만 해본 분
• 이직을 생각하는 분
• 은퇴에 대해 생각해볼 때가 된 분
정김경숙, 2024

🎢 Stories Related

• 전 까미노 걷고난 무렵 읽었는데, 공감되는 지점이 많았습니다.
• 저자가 말하는 육체노동의 즐거움이 특히 그러합니다.
• 까미노에서 느꼈던 묘한 평온의 이유이기도 하니까요.
🗨️ 좀 더 자세한 이야기

수필에 가까운 책이니, 요약하거나 전달할 내용이 많지는 않습니다. 하지만 지은이가 순간 순간 포착한 느낌과 생각이 읽는 이에게 의미로 다가옵니다. 화상회의로 구조조정을 전격 통지 당할 때의 생생한 느낌, 당장 출근할 곳이 없는 막막함, 그럼에도 씩씩하게 딱 한발만 내 디뎌보는 용기. 이런데서 묘한 공감과 위안을 얻습니다.

너무 사랑해 헤어지기 어려웠던 구글이지만, '회사가 먼저 손을 놓아줘' 고맙다는 마음을 먹는데서 이미 극복은 시작됩니다. layoff를 playoff로 생각하며 이참에 갭 이어를 갖기로 한 저자의 마음은 단단하고 슬기롭습니다.

몸통에 해당하는 글은 트레이더 조스, 스타벅스의 시간제 근무를 상세히 전하는데, 제법 재미납니다. 상상 이상으로 잘 체계화되어 있고, 시스템 안에 조직 문화까지 안배한 디자인은 제게 무척 인상적이었지요.
몸통에 해당하는 글은 트레이더 조스, 스타벅스의 시간제 근무를 상세히 전하는데, 제법 재미납니다. 

### Retrieve chunks with BM25 + similarity

In [37]:
import numpy as np
from rank_bm25 import BM25Okapi
from transformers import AutoTokenizer
from sklearn.metrics.pairwise import cosine_similarity
import re
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/masang/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [38]:
# Hugging Face tokenizer 불러오기
tokenizer = AutoTokenizer.from_pretrained("jhgan/ko-sbert-sts")

def hybrid_search(query, vector_db, documents, k=2, bm25_weight=0.5):
    """
    Hybrid search combining BM25 and cosine similarity with customizable weighting

    Parameters:
    - query: 검색 쿼리
    - vector_db: 코사인 유사도 검색용 벡터 데이터베이스
    - documents: 원본 문서 리스트
    - k: 반환할 결과 수
    - bm25_weight: BM25 가중치 (0-1), 나머지는 코사인 유사도 가중치

    Returns:
    - 최종 결과 리스트
    """
    # 1. 코사인 유사도 검색 (벡터 검색)
    cosine_results = vector_db.similarity_search_with_score(query, k=k*2)

    cosine_docs = [doc for doc, score in cosine_results]
    cosine_scores = [score for doc, score in cosine_results]

    max_score = max(cosine_scores)
    min_score = min(cosine_scores)
    score_range = max_score - min_score if max_score != min_score else 1
    normalized_cosine_scores = [1 - ((score - min_score) / score_range) for score in cosine_scores]

    # 2. BM25 준비 (Hugging Face tokenizer 기반)
    tokenized_docs = [tokenizer.tokenize(doc.page_content) for doc in documents]
    bm25 = BM25Okapi(tokenized_docs)

    tokenized_query = tokenizer.tokenize(query)
    bm25_scores = bm25.get_scores(tokenized_query)

    max_bm25 = max(bm25_scores)
    min_bm25 = min(bm25_scores)
    bm25_range = max_bm25 - min_bm25 if max_bm25 != min_bm25 else 1
    normalized_bm25_scores = [(score - min_bm25) / bm25_range for score in bm25_scores]

    # 3. 하이브리드 점수 계산
    hybrid_scores = []
    for i, doc in enumerate(documents):
        cosine_score = 0
        for j, cosine_doc in enumerate(cosine_docs):
            if doc.page_content == cosine_doc.page_content:
                cosine_score = normalized_cosine_scores[j]
                break
        hybrid_score = (bm25_weight * normalized_bm25_scores[i]) + ((1 - bm25_weight) * cosine_score)
        hybrid_scores.append((doc, hybrid_score))

    hybrid_results = sorted(hybrid_scores, key=lambda x: x[1], reverse=True)[:k]
    return [doc for doc, score in hybrid_results]

In [45]:
# 쿼리 실행
query = "AI의가치정렬(valuealignment)문제는구체적으로어떤우려를말하나요?"

# 하이브리드 검색 실행 (BM25 0%, 코사인 유사도 100%)
results = hybrid_search(query, loaded_vector_db, texts, k=3, bm25_weight=0.5)

# 결과 출력
for result in results:
    print(result.page_content, "\n\n")

AGI 문제는 악의를 염려하는게 아니라 그 성능이다. 공포장사가 아니라 안전공학 차원에서 제한을 미리 넣어두자(테그마크)

AI가 걱정되지만 잘 가르치면 괜찮을지도 파

기계보상에 답이 있다 (러셀)

대략 이런데, 위너의 생각을 계승한 학자들이 우려하는 건 가치정렬(value alignment)입니다.

첫째는 원숭이 발 문제 또는 미다스의 손 문제입니다. 시키는 것만 문자그대로 완수하여, 부작용이 문제가 되는 상황입니다. 암치료제를 찾는다고 인류를 대상으로 생체실험을 해버린다든지, 바다 산성을 낮추는 대신 대기를 망쳐놓는다든지입니다.

둘째는 영화적 공포입니다. 더 심화해서 AI가 자체의 욕망대로 움직이는 경우지요. 종이 클립 이슈라고 불립니다. 만일 종이 클립이 AI 기계 시스템의 입맛에 맞다고 치면, 문물을 관장하는 AI서버가 온 바다를 종이클립으로 메우지 않을 이유가 있을까 하는겁니다.

이에 대한 반론은 세가지 같습니다.

• 그렇게까지 기술을 발전시키기 쉽지 않다.
• 사람 모델링도 안되어 있으니, 종합적 인지모델 자체를 만들기 어렵다
• 어쨌든 사람 가르치듯 가르치면 된다
결국 AI 학습은 양육개념이라고 보면 더 잘 이해됩니다. 사람 아이도 잘 가르치기 어려운데, AI라고 쉽게 가르칠수 있겠습니까. 양자컴퓨팅을 만든 도이치는 진정한 AGI는 체스를 매번 이기도록 만든 때가 아니라 체스를 이기지 않기로 결정하는 능력이 생길 때라고 봅니다.

여기에 따라 나오는 개념이 자유의지입니다. 보상과 처벌을 내면화한 결과라고 생각할 수 있습니다. 자유의지는 윤리와 맞닿아 있고요. 


위너는 기술적으로 틀렸다 파

어차피 위너 시대에 지금 인공지능을 예측할 수도 없었거니와, 바로 뒤에 올 컴퓨터조차 제대로 예측하지 못했다. 그의 말은 시적이고 상상력을 자아내게 했을 뿐, 글자대로 해석하는건 무리다. 심지어 당대의 섀넌 이론도 이해하지 못한 말이다. 정보는 엔트로피일 뿐 의미가 아니다.

어쨌든 인류는 AI가 필요해 파

인간의 합리성을 믿을 필요가 있다. 결

### Hyde